<a href="https://colab.research.google.com/github/arjungoel7/Logo_Detection_Recognition/blob/master/LogoClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
%cd drive/My\ Drive/Deep\ Learning/qmul_toplogo10

In [0]:
import os
import numpy as np

In [0]:
dir = "masks"
temp = os.path.join(str(dir))

boxes = np.zeros((700, 4))
cnt = 0

for temp_dir_name in sorted(os.listdir(temp)):

    print(temp_dir_name)
    tmp = os.path.join(str(dir) + '/' + str(temp_dir_name))

    for fil in sorted(os.listdir(tmp)):
      f = open(str(dir) + '/' + str(temp_dir_name) + '/' + str(fil), "r")
      
      contents = f.read()
      # print(contents)

      w1 = int(contents.split()[0])
      h1 = int(contents.split()[1])
      w2 = int(contents.split()[2]) + w1
      h2 = int(contents.split()[3]) + h1
      # print(str(fil), w1, h1, w2, h2)

      boxes[cnt][0] = w1
      boxes[cnt][1] = h1
      boxes[cnt][2] = w2
      boxes[cnt][3] = h2

      cnt += 1

print(type(boxes))

# boxes = 

adidas0
chanel
gucci
hh
lacoste
mk
nike
prada
puma
supreme
<class 'numpy.ndarray'>


In [0]:
print(boxes)
boxes = np.reshape(boxes, (700, 4))
np.save('boxes', boxes)

[[567. 408. 730. 563.]
 [616.   7. 774. 118.]
 [283.  69. 369. 127.]
 ...
 [159. 226. 425. 339.]
 [293. 163. 368. 189.]
 [153. 160. 466. 267.]]


In [0]:
print(boxes.shape)

(700, 4)


In [0]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

labels = np.zeros(700, dtype = 'uint8')
images = np.zeros((700,200,200,3), dtype = 'uint8')
cnt = 0

dir = "jpg"
temp = os.path.join(str(dir))

for temp_dir_name in sorted(os.listdir(temp)):

    print(temp_dir_name)
    tmp = os.path.join(str(dir) + '/' + str(temp_dir_name))

    # print(sorted(os.listdir(tmp)))

    for image in sorted(os.listdir(tmp)):
      print(image)
      if image.endswith(".jpg") or image.endswith(".png"):
        im = cv2.imread(str(dir) + '/' + str(temp_dir_name) + '/' + str(image))

        box = boxes[cnt]
        print(box)
        box = box.reshape((1,4))
        w1 = int(box[0][0])
        h1 = int(box[0][1])
        w2 = int(box[0][2])
        h2 = int(box[0][3])
        print(w1, h1, w2, h2)
        
        # print(im.shape)
        # im = np.array(im)
        # print(im)

        tmpImg = np.zeros((h2 - h1, w2 - w1, 3))

        tmpImg = im[h1:h2, w1:w2, :]
        
        # print(tmpImg.shape)

        tmpImg = cv2.resize(tmpImg, (200, 200), interpolation = cv2.INTER_AREA)
        
        images[cnt] = tmpImg

        cnt += 1
        
      #   contents = f.read()
      #   # print(contents)
      #   w1 = int(contents.split()[0])
      #   w2 = int(contents.split()[2]) + w1
      #   h1 = int(contents.split()[1])
      #   h2 = int(contents.split()[3]) + h1
      #   print(str(fil), w1, w2, h1, h2)
  # print(temp_file_names)
  # for fil in temp_file_names:
  #     f = open(str(fil), "r")
  #     contents = f.read()
  #     # print(contents)
  #     w1 = int(contents.split()[0])
  #     w2 = int(contents.split()[2]) + w1
  #     h1 = int(contents.split()[1])
  #     h2 = int(contents.split()[3]) + h1
  #     print( w1, w2, h1, h2)
  #     # print(x1)

In [0]:
target_labels = np.zeros((700,10))
lbl = -1
for i in range(0,700):
  if i%70 == 0:
    lbl = lbl + 1
  target_labels[i][lbl] = 1
  

In [0]:
from sklearn.utils import shuffle
images, final_labels = shuffle(images, target_labels)

In [0]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Dense
from keras.models import Sequential

cnn_model = Sequential()

#after the input layer, add the first convolutional layer with 32 2x2-filters 
cnn_model.add(Conv2D (kernel_size = (2,2), filters = 32, 
                      input_shape=X_train.shape[1:], activation='relu'))
#add a max pooling layer with a 2x2 pooling window
cnn_model.add(MaxPooling2D(pool_size=2))
#add the second convolutional layer with 64 2x2-filters 
cnn_model.add(Conv2D(kernel_size = 2, filters = 64, activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=2))
#add the third convolutional layer with 128 2x2-filters 
cnn_model.add(Conv2D(kernel_size = 2, filters = 128, activation='relu'))
#add a dropout layer so that each node has a chance of 20% to be dropped when training
cnn_model.add(Dropout(0.2))
cnn_model.add(MaxPooling2D(pool_size = 2))
#add a global average pooling layer
cnn_model.add(GlobalAveragePooling2D())
#add the final fully connected output layer with 109 node for all 109 logo classes
cnn_model.add(Dense(10, activation = 'softmax'))

cnn_model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_49 (Conv2D)           (None, 199, 199, 32)      416       
_________________________________________________________________
max_pooling2d_49 (MaxPooling (None, 99, 99, 32)        0         
_________________________________________________________________
conv2d_50 (Conv2D)           (None, 98, 98, 64)        8256      
_________________________________________________________________
max_pooling2d_50 (MaxPooling (None, 49, 49, 64)        0         
_________________________________________________________________
conv2d_51 (Conv2D)           (None, 48, 48, 128)       32896     
_________________________________________________________________
dropout_17 (Dropout)         (None, 48, 48, 128)       0         
_________________________________________________________________
max_pooling2d_51 (MaxPooling (None, 24, 24, 128)     

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images, final_labels, test_size=0.33)

In [0]:
#Compile the model 
from keras.optimizers import adam

myopt = adam(lr=0.0001, decay=1e-6)

cnn_model.compile(optimizer=myopt, 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

#Train the model for 100 epochs
H = cnn_model.fit(X_train, y_train, epochs=100, verbose=1)

Epoch 1/100
469/469 [==============================] - 2s 4ms/step - loss: 8.2541 - acc: 0.1343
Epoch 2/100
469/469 [==============================] - 0s 816us/step - loss: 6.4479 - acc: 0.1770
Epoch 3/100
469/469 [==============================] - 0s 805us/step - loss: 4.5851 - acc: 0.2175
Epoch 4/100
469/469 [==============================] - 0s 813us/step - loss: 3.7834 - acc: 0.1748
Epoch 5/100
469/469 [==============================] - 0s 815us/step - loss: 2.8435 - acc: 0.2090
Epoch 6/100
469/469 [==============================] - 0s 822us/step - loss: 2.3383 - acc: 0.2260
Epoch 7/100
469/469 [==============================] - 0s 823us/step - loss: 2.1908 - acc: 0.2687
Epoch 8/100
469/469 [==============================] - 0s 807us/step - loss: 2.1339 - acc: 0.2623
Epoch 9/100
469/469 [==============================] - 0s 820us/step - loss: 2.0575 - acc: 0.2580
Epoch 10/100
469/469 [==============================] - 0s 812us/step - loss: 2.0250 - acc: 0.2985
Epoch 11/100
469/469 

In [0]:
cnn_model.evaluate(X_test,y_test, verbose = 1)

231/231 [==============================] - 1s 3ms/step


[1.7983300691043145, 0.4805194812935668]